In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU capability: {torch.cuda.get_device_capability(0)}")

PyTorch version: 2.5.1
CUDA available: True
CUDA version: 11.8
GPU name: NVIDIA GeForce RTX 3060 Laptop GPU
GPU capability: (8, 6)


In [3]:
%pip install av opencv-python numpy tqdm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import shutil
import random
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, vfx
from tqdm import tqdm
import mediapipe as mp

In [5]:
# --- 0. Configuration & Setup ---
RAW_VIDEO_DATASET_PATH = "SignLanguageDataset"  # <--- UPDATE THIS PATH
BASE_OUTPUT_PATH = "SignLanguage_Processed_Data" # <--- UPDATE THIS PATH (if desired)

SPLIT_DATA_PATH = os.path.join(BASE_OUTPUT_PATH, "video_splits")
AUGMENTED_TRAIN_VIDEOS_PATH = os.path.join(BASE_OUTPUT_PATH, "augmented_train_videos")
KEYPOINTS_OUTPUT_PATH = os.path.join(BASE_OUTPUT_PATH, "keypoints_for_model")

PREFERRED_GPU_ENCODER = 'h264_nvenc' # TRY NVIDIA ENCODER or set to "" for CPU only
CPU_FALLBACK_ENCODER = 'libx264'
AUDIO_CODEC = 'aac'

os.makedirs(BASE_OUTPUT_PATH, exist_ok=True)
os.makedirs(SPLIT_DATA_PATH, exist_ok=True)
os.makedirs(AUGMENTED_TRAIN_VIDEOS_PATH, exist_ok=True)
os.makedirs(KEYPOINTS_OUTPUT_PATH, exist_ok=True)

N_FRAMES_KEYPOINTS = 30
MOVEMENT_THRESHOLD_KEYPOINTS = 0.02

In [6]:
# --- 1. Dataset Splitting ---
def split_video_dataset(
    raw_dataset_path, output_split_path, train_ratio=0.7, val_ratio=0.15,
    test_ratio=0.15, random_seed=42, move_files=False
):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"
    random.seed(random_seed)
    if not os.path.exists(raw_dataset_path):
        print(f"Error: Raw dataset path '{raw_dataset_path}' does not exist.")
        return False

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_split_path, split), exist_ok=True)

    print(f"Starting dataset split from '{raw_dataset_path}' into '{output_split_path}'...")
    for class_name in os.listdir(raw_dataset_path):
        class_dir = os.path.join(raw_dataset_path, class_name)
        if not os.path.isdir(class_dir): continue

        video_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f)) and f.lower().endswith(('.mp4', '.avi', '.mov'))]
        if not video_files:
            print(f"Warning: No video files found in class '{class_name}'.")
            continue
        
        random.shuffle(video_files)
        n_total = len(video_files)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        n_test = n_total - n_train - n_val
        if n_test < 0: n_test = 0

        splits_data = {
            'train': video_files[:n_train],
            'val': video_files[n_train : n_train + n_val],
            'test': video_files[n_train + n_val :]
        }
        
        current_assigned_count = sum(len(v) for v in splits_data.values())
        if current_assigned_count < n_total:
            remaining_files = video_files[current_assigned_count:]
            if len(splits_data['test']) < n_test and test_ratio > 0:
                 splits_data['test'].extend(remaining_files)
            elif len(splits_data['val']) < n_val and val_ratio > 0:
                 splits_data['val'].extend(remaining_files)
            else:
                 splits_data['train'].extend(remaining_files)

        for split_name, files_in_split in splits_data.items():
            dest_class_dir = os.path.join(output_split_path, split_name, class_name)
            os.makedirs(dest_class_dir, exist_ok=True)
            for file_name in files_in_split:
                src_file = os.path.join(class_dir, file_name)
                dest_file = os.path.join(dest_class_dir, file_name)
                if move_files: shutil.move(src_file, dest_file)
                else: shutil.copy2(src_file, dest_file)
        
        print(f"Class '{class_name}' split: Train={len(splits_data['train'])}, Val={len(splits_data['val'])}, Test={len(splits_data['test'])}")
    print("✅ Dataset splitting complete.")
    return True


In [7]:
# --- 2. Video Augmentation ---
def rotate_video(clip, angle): return clip.fx(vfx.rotate, angle)
def zoom_video(clip, zoom_factor):
    w, h = clip.size
    new_w, new_h = int(w / zoom_factor), int(h / zoom_factor)
    x1, y1 = (w - new_w) // 2, (h - new_h) // 2
    return clip.fx(vfx.crop, x1=x1, y1=y1, width=new_w, height=new_h).resize(clip.size)
def flip_horizontal_video(clip): return clip.fx(vfx.mirror_x)
def add_gaussian_noise_vfx(clip, sigma=25):
    def apply_noise(gf, t):
        f = gf(t); n = np.random.normal(0, sigma, f.shape).astype(np.uint8); return cv2.add(f, n)
    return clip.fl(apply_noise)
def change_brightness_contrast_vfx(clip, bf=1.2, cf=1.2):
    def apply_bc(gf,t): f=gf(t); a=cf; b=(bf-1)*127; return cv2.convertScaleAbs(f,alpha=a,beta=b)
    return clip.fl(apply_bc)
def translate_video_vfx(clip, dx, dy):
    def apply_trans(gf,t): f=gf(t); M=np.float32([[1,0,dx],[0,1,dy]]); return cv2.warpAffine(f,M,(f.shape[1],f.shape[0]))
    return clip.fl(apply_trans)
def add_blur_vfx(clip, ks=(5,5)):
    def apply_blur(gf,t): return cv2.GaussianBlur(gf(t),ks,0)
    return clip.fl(apply_blur)
def color_jitter_video(clip, br=0.2, co=0.2, sa=0.2, hu=0.1):
    def apply_cj(gf,t):
        f=gf(t); hsv=cv2.cvtColor(f,cv2.COLOR_RGB2HSV); h,s,v_=cv2.split(hsv)
        s=np.clip(s*(1+random.uniform(-sa,sa)),0,255).astype(np.uint8)
        h=np.clip(h+random.uniform(-hu*180,hu*180),0,179).astype(np.uint8)
        fm=cv2.cvtColor(cv2.merge([h,s,v_]),cv2.COLOR_HSV2RGB)
        a=1.0+random.uniform(-co,co); b_v=random.uniform(-br,br)*255
        return cv2.convertScaleAbs(fm,alpha=a,beta=b_v)
    return clip.fl(apply_cj)

AUGMENTATIONS = {
    "rotation": lambda clip: rotate_video(clip, random.uniform(-10, 10)),
    "zoom": lambda clip: zoom_video(clip, random.uniform(1.05, 1.2)),
    # "flip_horizontal": lambda clip: flip_horizontal_video(clip), # Often not useful for SL
    "gaussian_noise": lambda clip: add_gaussian_noise_vfx(clip, sigma=random.uniform(10, 20)), # Reduced sigma
    "brightness_contrast": lambda clip: change_brightness_contrast_vfx(clip, bf=random.uniform(0.9, 1.1), cf=random.uniform(0.9, 1.1)), # Reduced factors
    "translation": lambda clip: translate_video_vfx(clip, dx=random.randint(-10, 10), dy=random.randint(-10, 10)), # Reduced translation
    "blur": lambda clip: add_blur_vfx(clip, ks=(random.choice([3, 5]), random.choice([3, 5]))),
    "color_jitter": lambda clip: color_jitter_video(clip, br=0.1, co=0.1, sa=0.1, hu=0.05), # Reduced jitter
}

def augment_video_set(train_video_input_path, augmented_output_path):
    os.makedirs(augmented_output_path, exist_ok=True)
    print(f"Starting video augmentation for videos in: {train_video_input_path}")
    total_processed, total_augmented = 0, 0
    for class_name in os.listdir(train_video_input_path):
        class_path = os.path.join(train_video_input_path, class_name)
        if not os.path.isdir(class_path): continue
        output_class_path = os.path.join(augmented_output_path, class_name)
        os.makedirs(output_class_path, exist_ok=True)
        print(f"\nAugmenting class: {class_name}")
        video_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        for video_file in tqdm(video_files, desc=f"Videos in {class_name}"):
            video_path = os.path.join(class_path, video_file); base_name, ext = os.path.splitext(video_file)
            output_original_path = os.path.join(output_class_path, video_file)
            try: shutil.copy2(video_path, output_original_path); total_processed += 1
            except Exception as e: print(f"Error copying {video_file}: {e}"); continue
            for aug_name, aug_func in AUGMENTATIONS.items():
                output_filename = f"{base_name}_{aug_name}{ext}"
                output_aug_path = os.path.join(output_class_path, output_filename)
                if os.path.exists(output_aug_path): continue # Skip if already augmented
                try:
                    clip = VideoFileClip(video_path); augmented_clip = aug_func(clip.copy())
                    encoder_params = {'codec': CPU_FALLBACK_ENCODER, 'audio_codec': AUDIO_CODEC, 'logger': None, 'threads': 4, 'preset': 'ultrafast'}
                    if PREFERRED_GPU_ENCODER:
                        try:
                            gpu_params = encoder_params.copy(); gpu_params['codec'] = PREFERRED_GPU_ENCODER; del gpu_params['preset']
                            augmented_clip.write_videofile(output_aug_path, **gpu_params)
                        except Exception: augmented_clip.write_videofile(output_aug_path, **encoder_params)
                    else: augmented_clip.write_videofile(output_aug_path, **encoder_params)
                    clip.close(); augmented_clip.close(); total_augmented += 1
                except Exception as e:
                    print(f"\nError: {aug_name} for {video_file}: {e}")
                    if os.path.exists(output_aug_path): os.remove(output_aug_path)
    print(f"\n--- Video Augmentation Complete ---\nOriginals: {total_processed}, Augmentations: {total_augmented}")


In [8]:
# --- 3. Keypoint Extraction ---
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def extract_holistic_landmarks(results):
    def get_coords(lc, es):
        if lc and lc.landmark: return np.array([[lm.x,lm.y,lm.z] for lm in lc.landmark], dtype=np.float32)
        return np.zeros((es,3),dtype=np.float32)
    return np.concatenate([get_coords(results.pose_landmarks,33), get_coords(results.face_landmarks,468),
                           get_coords(results.left_hand_landmarks,21), get_coords(results.right_hand_landmarks,21)], axis=0)
def get_hand_center(hl): return np.array([hl.landmark[0].x,hl.landmark[0].y,hl.landmark[0].z],dtype=np.float32) if hl and hl.landmark else None
def hands_moved(pl,pr,cl,cr,th):
    if any(x is None for x in [pl,pr,cl,cr]): return False
    return (np.linalg.norm(cl-pl)>th or np.linalg.norm(cr-pr)>th)

def process_video_for_keypoints(video_path, n_frames=N_FRAMES_KEYPOINTS, movement_threshold=MOVEMENT_THRESHOLD_KEYPOINTS):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): print(f"Error: Vid {video_path}"); return np.zeros((n_frames,543,3),dtype=np.float32)
    seq_kps, m_start, plh, prh, f_cnt_mov, p_f_cnt = [], False, None, None, 0, 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        p_f_cnt+=1; img_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB); results=holistic_model.process(img_rgb)
        clh,crh=get_hand_center(results.left_hand_landmarks),get_hand_center(results.right_hand_landmarks)
        if not m_start:
            if hands_moved(plh,prh,clh,crh,movement_threshold): m_start=True
            plh,prh=clh,crh; continue
        f_cnt_mov+=1
        if f_cnt_mov%2==0: seq_kps.append(extract_holistic_landmarks(results))
        if len(seq_kps)==n_frames: break
    cap.release()
    if not seq_kps:
        print(f"Warn: No kps for '{video_path}'. Frames: {p_f_cnt}.")
        return np.zeros((n_frames,543,3),dtype=np.float32)
    if len(seq_kps)<n_frames: seq_kps.extend([seq_kps[-1]]*(n_frames-len(seq_kps)))
    return np.array(seq_kps[:n_frames])

def extract_keypoints_from_video_set(video_set_input_path, keypoints_root_output_dir, split_name):
    print(f"\nKeypoint extraction for '{split_name}' set from: {video_set_input_path}")
    if not os.path.exists(video_set_input_path): print(f"Error: Path '{video_set_input_path}' DNE."); return
    for class_name in os.listdir(video_set_input_path):
        videos_in_class_path = os.path.join(video_set_input_path, class_name)
        if not os.path.isdir(videos_in_class_path): continue
        keypoints_out_class_split = os.path.join(keypoints_root_output_dir,class_name,split_name)
        os.makedirs(keypoints_out_class_split,exist_ok=True)
        print(f"Processing class: {class_name} ({split_name})")
        vfs = [f for f in os.listdir(videos_in_class_path) if f.lower().endswith(('.mp4','.avi','.mov'))]
        for vf in tqdm(vfs, desc=f"Keypoints {class_name} ({split_name})"):
            vfp=os.path.join(videos_in_class_path,vf); bn,_=os.path.splitext(vf)
            onfp=os.path.join(keypoints_out_class_split,f"{bn}.npy")
            if os.path.exists(onfp): continue
            np.save(onfp, process_video_for_keypoints(vfp))
    print(f"✅ Keypoint extraction for '{split_name}' set complete.")


In [9]:
# --- Main Execution ---
if __name__ == "__main__":
    # Step 1: Split
    print("--- STEP 1: Splitting Dataset ---")
    train_dir = os.path.join(SPLIT_DATA_PATH, "train")
    if not (os.path.exists(train_dir) and os.listdir(train_dir)):
        if not split_video_dataset(RAW_VIDEO_DATASET_PATH, SPLIT_DATA_PATH):
            print("Halting due to error in video dataset splitting."); exit()
    else: print(f"Video splits found at '{SPLIT_DATA_PATH}'. Skipping.")

    # Step 2: Augment
    print("\n--- STEP 2: Augmenting Training Videos ---")
    train_split_path_for_aug = os.path.join(SPLIT_DATA_PATH, "train")
    aug_train_dir_has_content = os.path.exists(AUGMENTED_TRAIN_VIDEOS_PATH) and any(os.scandir(AUGMENTED_TRAIN_VIDEOS_PATH))
    if not aug_train_dir_has_content :
         if os.path.exists(train_split_path_for_aug) and any(os.scandir(train_split_path_for_aug)):
            augment_video_set(train_split_path_for_aug, AUGMENTED_TRAIN_VIDEOS_PATH)
         else: print(f"Skipping augmentation: Training split path '{train_split_path_for_aug}' empty/DNE.")
    else: print(f"Augmented videos found at '{AUGMENTED_TRAIN_VIDEOS_PATH}'. Skipping.")

    # Step 3: Extract Keypoints
    print("\n--- STEP 3: Extracting Keypoints ---")
    kp_train_dir_example = os.path.join(KEYPOINTS_OUTPUT_PATH, (os.listdir(RAW_VIDEO_DATASET_PATH)[0] if os.path.exists(RAW_VIDEO_DATASET_PATH) and os.listdir(RAW_VIDEO_DATASET_PATH) else "dummy"), "train")
    
    keypoints_exist = os.path.exists(kp_train_dir_example) and any(os.scandir(kp_train_dir_example))
    
    if not keypoints_exist:
        source_for_train_kp = AUGMENTED_TRAIN_VIDEOS_PATH if (os.path.exists(AUGMENTED_TRAIN_VIDEOS_PATH) and any(os.scandir(AUGMENTED_TRAIN_VIDEOS_PATH))) else train_split_path_for_aug
        if os.path.exists(source_for_train_kp) and any(os.scandir(source_for_train_kp)):
            print(f"Extracting train keypoints from: {source_for_train_kp}")
            extract_keypoints_from_video_set(source_for_train_kp, KEYPOINTS_OUTPUT_PATH, "train")
        else: print(f"Source for train keypoints ('{source_for_train_kp}') empty/DNE.")

        val_split_path_for_kp = os.path.join(SPLIT_DATA_PATH, "val")
        if os.path.exists(val_split_path_for_kp) and any(os.scandir(val_split_path_for_kp)):
            extract_keypoints_from_video_set(val_split_path_for_kp, KEYPOINTS_OUTPUT_PATH, "val")
        else: print(f"Val split path '{val_split_path_for_kp}' empty/DNE for keypoints.")

        test_split_path_for_kp = os.path.join(SPLIT_DATA_PATH, "test")
        if os.path.exists(test_split_path_for_kp) and any(os.scandir(test_split_path_for_kp)):
            extract_keypoints_from_video_set(test_split_path_for_kp, KEYPOINTS_OUTPUT_PATH, "test")
        else: print(f"Test split path '{test_split_path_for_kp}' empty/DNE for keypoints.")
    else:
        print(f"Keypoints seem extracted in '{KEYPOINTS_OUTPUT_PATH}'. Skipping.")
        
    if 'holistic_model' in globals(): holistic_model.close() 
    print("\n--- ALL PREPROCESSING STEPS ATTEMPTED ---")

--- STEP 1: Splitting Dataset ---
Starting dataset split from 'SignLanguageDataset' into 'SignLanguage_Processed_Data\video_splits'...
Class 'clavier' split: Train=34, Val=7, Test=8
Class 'disque_dur' split: Train=35, Val=7, Test=8
Class 'ordinateur' split: Train=35, Val=7, Test=8
Class 'souris' split: Train=35, Val=7, Test=8
✅ Dataset splitting complete.

--- STEP 2: Augmenting Training Videos ---
Starting video augmentation for videos in: SignLanguage_Processed_Data\video_splits\train

Augmenting class: clavier


Videos in clavier:   0%|          | 0/34 [00:00<?, ?it/s]

Moviepy - Building video SignLanguage_Processed_Data\augmented_train_videos\clavier\clavier1_rotation.mp4.
MoviePy - Writing audio in clavier1_rotationTEMP_MPY_wvf_snd.mp3


Videos in clavier:   0%|          | 0/34 [00:01<?, ?it/s]

MoviePy - Done.
Moviepy - Writing video SignLanguage_Processed_Data\augmented_train_videos\clavier\clavier1_rotation.mp4

Moviepy - Building video SignLanguage_Processed_Data\augmented_train_videos\clavier\clavier1_rotation.mp4.
MoviePy - Writing audio in clavier1_rotationTEMP_MPY_wvf_snd.mp3


...

Warn: No kps for 'SignLanguage_Processed_Data\augmented_train_videos\souris\souris49.mp4'. Frames: 182.


Keypoints souris (train): 100%|██████████| 35/35 [04:13<00:00,  7.23s/it]


✅ Keypoint extraction for 'train' set complete.

Keypoint extraction for 'val' set from: SignLanguage_Processed_Data\video_splits\val
Processing class: clavier (val)


Keypoints clavier (val):  14%|█▍        | 1/7 [00:09<00:58,  9.77s/it]


KeyboardInterrupt: 

# MSKA: Multi-Stream Keypoint-based Action Recognition for Sign Language (Hands Only)

### 1. Setup and Imports

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
from tqdm.notebook import tqdm 
from torchmetrics import F1Score, Accuracy
import matplotlib.pyplot as plt
import seaborn as sns
from einops import rearrange 

DATA_DIR = "SignLanguage_Processed_Data/keypoints_for_model"  # <--- UPDATE THIS PATH
NUM_CLASSES = 4  # <--- UPDATE THIS for "clavier, disque dur, ordinateur, souris"
BATCH_SIZE = 16
NUM_EPOCHS = 50 
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 1e-4
D_MODEL = 128 
N_HEAD = 4    
NUM_LAYERS = 2 

LEFT_HAND_INDICES = slice(501, 522)
RIGHT_HAND_INDICES = slice(522, 543)
NUM_HAND_KEYPOINTS = 21
KEYPOINT_FEATURES = 3 

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

### 2. Dataset Class and Data Loading Utilities

In [ ]:
class SignLanguageKeypointsDataset(Dataset):
    def __init__(self, data_samples, labels, max_frames):
        self.data_samples = data_samples
        self.labels = labels
        self.max_frames = max_frames

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file_path = self.data_samples[idx]
        label = self.labels[idx]
        try:
            holistic_data = np.load(file_path)
            # Basic sanity check for loaded data shape
            if not (holistic_data.ndim == 3 and holistic_data.shape[1] == 543 and holistic_data.shape[2] == KEYPOINT_FEATURES and holistic_data.shape[0] > 0):
                print(f"Warning: File {file_path} has unexpected shape {holistic_data.shape} or zero frames. Returning zeros.")
                holistic_data = np.zeros((self.max_frames, 543, KEYPOINT_FEATURES), dtype=np.float32)
        except Exception as e:
            print(f"Error loading {file_path}: {e}. Returning zeros.")
            holistic_data = np.zeros((self.max_frames, 543, KEYPOINT_FEATURES), dtype=np.float32)

        left_hand_kps = holistic_data[:, LEFT_HAND_INDICES, :]
        right_hand_kps = holistic_data[:, RIGHT_HAND_INDICES, :]

        left_hand_kps = self.pad_or_truncate_stream(left_hand_kps, NUM_HAND_KEYPOINTS)
        right_hand_kps = self.pad_or_truncate_stream(right_hand_kps, NUM_HAND_KEYPOINTS)

        return (
            torch.tensor(left_hand_kps, dtype=torch.float32),
            torch.tensor(right_hand_kps, dtype=torch.float32),
            torch.tensor(label, dtype=torch.long),
        )

    def pad_or_truncate_stream(self, stream, num_keypoints_per_stream):
        current_frames = stream.shape[0]
        if current_frames == 0: 
            return np.zeros((self.max_frames, num_keypoints_per_stream, KEYPOINT_FEATURES), dtype=np.float32)
        if current_frames == self.max_frames:
            return stream
        elif current_frames < self.max_frames:
            padding_shape = (self.max_frames - current_frames, num_keypoints_per_stream, KEYPOINT_FEATURES)
            padding = np.zeros(padding_shape, dtype=stream.dtype)
            return np.concatenate([stream, padding], axis=0)
        else:
            return stream[:self.max_frames, :, :]

### 3. Model Architecture (Hands Only)

In [ ]:
class SpatialAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
    def forward(self, x):
        batch, frames, num_kps, d_model_feat = x.shape
        x_reshaped = x.reshape(batch * frames, num_kps, d_model_feat)
        Q, K, V = self.query(x_reshaped), self.key(x_reshaped), self.value(x_reshaped)
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (d_model_feat**0.5)
        attn_probs = F.softmax(attn_scores, dim=-1)
        out = torch.matmul(attn_probs, V)
        return out.reshape(batch, frames, num_kps, d_model_feat)

class TemporalTransformer(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward_factor=4):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                               dim_feedforward=d_model * dim_feedforward_factor, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)
    def forward(self, x):
        x = self.transformer_encoder(x)
        return self.norm(x)

class ConvTransformerBlock(nn.Module):
    def __init__(self, d_model, nhead, num_transformer_layers):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv1d(d_model, d_model * 2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(d_model * 2, d_model, kernel_size=3, padding=1))
        self.norm_after_conv = nn.LayerNorm(d_model)
        self.transformer = TemporalTransformer(d_model, nhead, num_transformer_layers)
    def forward(self, x):
        x_permuted = x.permute(0, 2, 1)
        conv_out = self.conv_block(x_permuted)
        conv_out_permuted = conv_out.permute(0, 2, 1)
        x_residual = x + conv_out_permuted 
        x_normed = self.norm_after_conv(x_residual) 
        return self.transformer(x_normed)

class HandStreamProcessor(nn.Module):
    def __init__(self, in_keypoint_features, num_keypoints, d_model, nhead, num_layers):
        super().__init__()
        self.projection = nn.Sequential(nn.Linear(in_keypoint_features, d_model), nn.ReLU(), nn.LayerNorm(d_model))
        self.spatial_attn = SpatialAttention(d_model)
        self.temporal_conv_transformer = ConvTransformerBlock(d_model, nhead, num_layers)
        self.temporal_attn_pool = nn.Linear(d_model, 1) 
    def forward(self, x):
        x = self.projection(x)
        x = self.spatial_attn(x)
        x = x.mean(dim=2)
        x = self.temporal_conv_transformer(x)
        attn_weights = F.softmax(self.temporal_attn_pool(x).squeeze(-1), dim=-1)
        return torch.sum(x * attn_weights.unsqueeze(-1), dim=1)

class SignLanguageHandsModel(nn.Module):
    def __init__(self, num_classes, in_keypoint_features, num_hand_keypoints, d_model, nhead, num_layers):
        super().__init__()
        self.left_hand_processor = HandStreamProcessor(in_keypoint_features, num_hand_keypoints, d_model, nhead, num_layers)
        self.right_hand_processor = HandStreamProcessor(in_keypoint_features, num_hand_keypoints, d_model, nhead, num_layers)
        self.classifier = nn.Sequential(nn.Linear(d_model * 2, d_model), nn.ReLU(), nn.Dropout(0.3), nn.Linear(d_model, num_classes))
    def forward(self, left_hand_input, right_hand_input):
        left_out = self.left_hand_processor(left_hand_input)
        right_out = self.right_hand_processor(right_hand_input)
        combined_features = torch.cat([left_out, right_out], dim=-1)
        return self.classifier(combined_features)

IN_KEYPOINT_FEATURES = KEYPOINT_FEATURES
NUM_KP_PER_HAND = NUM_HAND_KEYPOINTS

### 4. Loss Function (Balanced Focal Loss)

In [ ]:
class BalancedFocalLoss(nn.Module):
    def __init__(self, class_counts, gamma=2.0, beta=0.9999):
        super().__init__()
        self.gamma = gamma
        if not isinstance(class_counts, torch.Tensor):
            class_counts = torch.tensor(class_counts, dtype=torch.float)
        effective_num = 1.0 - torch.pow(beta, class_counts)
        weights = (1.0 - beta) / torch.clamp(effective_num, min=1e-8) 
        weights = weights / weights.sum() * len(class_counts) 
        self.register_buffer('weights', weights)
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_term = (1 - pt) ** self.gamma
        alpha = self.weights[targets].to(inputs.device) 
        balanced_focal_loss = alpha * focal_term * ce_loss
        return balanced_focal_loss.mean()

### 5. Training and Validation Loop Definition

In [ ]:
def train_model(model, current_train_loader, current_val_loader, num_classes, device, num_epochs, learning_rate, weight_decay, current_y_train_labels):
    train_f1_metric = F1Score(task='multiclass', num_classes=num_classes, average='macro').to(device)
    val_f1_metric = F1Score(task='multiclass', num_classes=num_classes, average='macro').to(device)
    train_accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes).to(device)
    val_accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes).to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=7, factor=0.5, verbose=True) # Increased patience

    if len(current_y_train_labels) == 0:
        raise ValueError("current_y_train_labels is empty.")
    class_counts_np = np.bincount(current_y_train_labels, minlength=num_classes)
    class_counts_tensor = torch.tensor(class_counts_np, dtype=torch.float).to(device)
    criterion = BalancedFocalLoss(class_counts=class_counts_tensor, gamma=2.0)

    best_val_f1 = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': [], 'train_acc': [], 'val_acc': []}

    print("\n--- Starting Training ---")
    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0.0
        train_pbar = tqdm(current_train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', leave=False)
        
        for left_kps, right_kps, labels in train_pbar:
            left_kps, right_kps, labels = left_kps.to(device), right_kps.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(left_kps, right_kps)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            epoch_train_loss += loss.item()
            train_f1_metric.update(outputs, labels)
            train_accuracy_metric.update(outputs, labels)
            train_pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        avg_train_loss = epoch_train_loss / len(current_train_loader)
        epoch_train_f1 = train_f1_metric.compute().item()
        epoch_train_acc = train_accuracy_metric.compute().item()
        train_f1_metric.reset(); train_accuracy_metric.reset()

        model.eval()
        epoch_val_loss = 0.0
        val_pbar = tqdm(current_val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]', leave=False)
        with torch.no_grad():
            for left_kps, right_kps, labels in val_pbar:
                left_kps, right_kps, labels = left_kps.to(device), right_kps.to(device), labels.to(device)
                outputs = model(left_kps, right_kps)
                loss = criterion(outputs, labels)
                epoch_val_loss += loss.item()
                val_f1_metric.update(outputs, labels)
                val_accuracy_metric.update(outputs, labels)
                val_pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        avg_val_loss = epoch_val_loss / len(current_val_loader)
        epoch_val_f1 = val_f1_metric.compute().item()
        epoch_val_acc = val_accuracy_metric.compute().item()
        val_f1_metric.reset(); val_accuracy_metric.reset()

        history['train_loss'].append(avg_train_loss); history['val_loss'].append(avg_val_loss)
        history['train_f1'].append(epoch_train_f1); history['val_f1'].append(epoch_val_f1)
        history['train_acc'].append(epoch_train_acc); history['val_acc'].append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} Summary:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train F1: {epoch_train_f1:.4f}, Train Acc: {epoch_train_acc:.4f}")
        print(f"  Val Loss:   {avg_val_loss:.4f}, Val F1:   {epoch_val_f1:.4f}, Val Acc:   {epoch_val_acc:.4f}")
        scheduler.step(epoch_val_f1)
        if epoch_val_f1 > best_val_f1:
            best_val_f1 = epoch_val_f1
            torch.save(model.state_dict(), 'best_sign_model_hands_only.pth')
            print(f"  🔥 New best model saved with Val F1: {best_val_f1:.4f}")

    print("--- Training Finished ---")
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1); plt.plot(history['train_loss'], label='Train Loss'); plt.plot(history['val_loss'], label='Val Loss'); plt.title('Loss Evolution'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.subplot(1, 3, 2); plt.plot(history['train_f1'], label='Train F1'); plt.plot(history['val_f1'], label='Val F1'); plt.title('F1 Score Evolution'); plt.xlabel('Epoch'); plt.ylabel('F1 Score'); plt.legend(); plt.grid(True)
    plt.subplot(1, 3, 3); plt.plot(history['train_acc'], label='Train Accuracy'); plt.plot(history['val_acc'], label='Val Accuracy'); plt.title('Accuracy Evolution'); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True)
    plt.tight_layout(); plt.savefig('sign_language_training_hands_only.png'); plt.show()

    model.load_state_dict(torch.load('best_sign_model_hands_only.pth'))
    return model, history

### 6. Evaluation Function Definition

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

def evaluate_model(model, current_test_loader, device, current_label_mapping, num_classes_eval):
    model.eval()
    all_preds = []
    all_labels = []
    
    test_pbar = tqdm(current_test_loader, desc='Evaluating on Test Set', leave=False)
    with torch.no_grad():
        for left_kps, right_kps, labels in test_pbar:
            left_kps, right_kps = left_kps.to(device), right_kps.to(device)
            outputs = model(left_kps, right_kps)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    unique_labels_in_test = sorted(list(set(all_labels)))
    target_names_report = [name for name, idx in sorted(current_label_mapping.items(), key=lambda item: item[1]) if idx in unique_labels_in_test]
    cm_target_names = [name for name, idx in sorted(current_label_mapping.items(), key=lambda item: item[1])]
    cm_labels = list(range(num_classes_eval))

    print("\n--- Test Set Evaluation ---")
    print("\n📊 Classification Report:")
    if not target_names_report and unique_labels_in_test:
        print("Warning: target_names_report is empty, using stringified labels for report.")
        target_names_report = [str(l) for l in unique_labels_in_test]
    
    if not unique_labels_in_test:
        print("No labels found in the test set for evaluation.")
        return {'accuracy': 0, 'f1_macro': 0, 'f1_weighted': 0, 'classification_report': "No labels", 'confusion_matrix': np.array([])}

    print(classification_report(all_labels, all_preds, labels=unique_labels_in_test, target_names=target_names_report, digits=4, zero_division=0))
    
    test_accuracy = accuracy_score(all_labels, all_preds)
    test_f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    test_f1_weighted = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print(f"\n🔍 Test Accuracy: {test_accuracy:.4f}")
    print(f"🔍 Test F1-Score (Macro): {test_f1_macro:.4f}")
    print(f"🔍 Test F1-Score (Weighted): {test_f1_weighted:.4f}")

    cm = confusion_matrix(all_labels, all_preds, labels=cm_labels)
    plt.figure(figsize=(max(10, num_classes_eval // 1.5), max(8, num_classes_eval // 2)))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=cm_target_names, yticklabels=cm_target_names)
    plt.title("Confusion Matrix"); plt.xlabel("Predicted"); plt.ylabel("True")
    plt.xticks(rotation=45, ha='right'); plt.yticks(rotation=0)
    plt.tight_layout(); plt.savefig('confusion_matrix_hands_only.png'); plt.show()
    
    return {'accuracy': test_accuracy, 'f1_macro': test_f1_macro, 'f1_weighted': test_f1_weighted}

### 7. Main Execution Block (Data Loading, Training, Evaluation)

In [ ]:
# This cell groups the main logic for notebook execution.
# It relies on variables defined in the first cell (DATA_DIR, NUM_CLASSES, etc.)
# and class/function definitions from the cells above.

# --- Load data paths, labels, and create DataLoaders (copied from cell 2 for self-containment) ---
all_file_paths = []
all_labels = []
label_mapping = {} # Will be populated here
current_label_idx = 0
frame_counts = []

print(f"Loading data from: {DATA_DIR}")
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"DATA_DIR not found: {DATA_DIR}. Please run preprocessing script first.")
    
for class_name in sorted(os.listdir(DATA_DIR)):
    class_path = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_path):
        continue
    if class_name not in label_mapping:
        label_mapping[class_name] = current_label_idx
        current_label_idx += 1
    class_label = label_mapping[class_name]
    for split_name in sorted(os.listdir(class_path)):
        split_path = os.path.join(class_path, split_name)
        if not os.path.isdir(split_path):
            continue
        for file_name in os.listdir(split_path):
            if file_name.endswith(".npy"):
                file_path = os.path.join(split_path, file_name)
                all_file_paths.append(file_path)
                all_labels.append(class_label)
                try:
                    data = np.load(file_path)
                    if data.ndim == 3 and data.shape[1] == 543 and data.shape[2] == KEYPOINT_FEATURES and data.shape[0] > 0:
                       frame_counts.append(data.shape[0])
                    else: 
                       print(f"Warning: File {file_path} has unexpected shape {data.shape if hasattr(data, 'shape') else 'N/A'} or zero frames. Marking as problematic.")
                       frame_counts.append(0) # Mark as problematic
                except Exception as e:
                    print(f"Warning: Could not load {file_path} to get frame count: {e}")
                    frame_counts.append(0)

if not all_file_paths:
    raise ValueError(f"No .npy files found in {DATA_DIR}.")

valid_frame_counts = [fc for fc in frame_counts if fc > 0]
if valid_frame_counts:
    MAX_FRAMES = int(np.percentile(valid_frame_counts, 95))
    print(f"Using MAX_FRAMES = {MAX_FRAMES} (95th percentile of {len(valid_frame_counts)} valid files)")
else:
    MAX_FRAMES = 30 
    print(f"Warning: No valid frame counts. Using default MAX_FRAMES = {MAX_FRAMES}")

X_paths = np.array(all_file_paths)
y_labels_all = np.array(all_labels)

sss_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_val_indices, test_indices = next(sss_test.split(X_paths, y_labels_all))
X_train_val_paths, X_test_paths = X_paths[train_val_indices], X_paths[test_indices]
y_train_val_labels, y_test_labels = y_labels_all[train_val_indices], y_labels_all[test_indices]

sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_indices, val_indices = next(sss_val.split(X_train_val_paths, y_train_val_labels))
X_train_paths, X_val_paths = X_train_val_paths[train_indices], X_train_val_paths[val_indices]
y_train_labels, y_val_labels = y_train_val_labels[train_indices], y_train_val_labels[val_indices]

print(f"\nDataset splits:")
print(f"Train samples: {len(X_train_paths)}")
print(f"Validation samples: {len(X_val_paths)}")
print(f"Test samples: {len(X_test_paths)}")

train_dataset = SignLanguageKeypointsDataset(X_train_paths, y_train_labels, MAX_FRAMES)
val_dataset = SignLanguageKeypointsDataset(X_val_paths, y_val_labels, MAX_FRAMES)
test_dataset = SignLanguageKeypointsDataset(X_test_paths, y_test_labels, MAX_FRAMES)

DEBUG_DATALOADER = False 
NUM_WORKERS_CFG = 0 if DEBUG_DATALOADER or os.name != 'posix' else 2 

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS_CFG, pin_memory=True if DEVICE.type == 'cuda' else False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS_CFG, pin_memory=True if DEVICE.type == 'cuda' else False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS_CFG, pin_memory=True if DEVICE.type == 'cuda' else False)

print("\nClass distribution in splits:")
for split_name, labels_in_split_arg in [('Train', y_train_labels), ('Validation', y_val_labels), ('Test', y_test_labels)]:
    counts = Counter(labels_in_split_arg)
    print(f"{split_name}:")
    for label_idx, count in sorted(counts.items()):
        class_name_found = [name for name, idx in label_mapping.items() if idx == label_idx]
        if class_name_found:
            class_name = class_name_found[0]
            print(f"  {class_name} (ID {label_idx}): {count}")
        else:
            print(f"  Unknown Label ID {label_idx}: {count}")

# --- Model Instantiation and Training Execution ---
model_instance = SignLanguageHandsModel(
    num_classes=NUM_CLASSES, 
    in_keypoint_features=IN_KEYPOINT_FEATURES, 
    num_hand_keypoints=NUM_KP_PER_HAND, 
    d_model=D_MODEL, 
    nhead=N_HEAD, 
    num_layers=NUM_LAYERS
).to(DEVICE)

print(f"Model instantiated with {sum(p.numel() for p in model_instance.parameters() if p.requires_grad)} trainable parameters.")

trained_model_instance = None
history_data = None
if train_loader and val_loader and len(y_train_labels) > 0:
    trained_model_instance, history_data = train_model(
        model=model_instance,
        current_train_loader=train_loader,
        current_val_loader=val_loader,
        num_classes=NUM_CLASSES,
        device=DEVICE,
        num_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        current_y_train_labels=y_train_labels 
    )
else:
    print("Dataloaders or y_train_labels not defined/empty. Skipping training.")

# --- Evaluation on Test Set ---
model_for_eval = None
if trained_model_instance is not None:
    print("\nEvaluating the model trained in this session.")
    model_for_eval = trained_model_instance
elif os.path.exists('best_sign_model_hands_only.pth'):
    print("\nLoading best saved model ('best_sign_model_hands_only.pth') for evaluation.")
    model_for_eval = SignLanguageHandsModel(
        num_classes=NUM_CLASSES, in_keypoint_features=IN_KEYPOINT_FEATURES, 
        num_hand_keypoints=NUM_KP_PER_HAND, d_model=D_MODEL, 
        nhead=N_HEAD, num_layers=NUM_LAYERS
    ).to(DEVICE)
    try:
        model_for_eval.load_state_dict(torch.load('best_sign_model_hands_only.pth', map_location=DEVICE))
    except Exception as e:
        print(f"Error loading saved model: {e}")
        model_for_eval = None
else:
    print("No trained model from this session and no 'best_sign_model_hands_only.pth' found. Skipping evaluation.")

if model_for_eval and test_loader is not None:
    if label_mapping:
        evaluation_results = evaluate_model(model_for_eval, test_loader, DEVICE, label_mapping, NUM_CLASSES)
    else:
        print("Error: label_mapping not defined or empty. Cannot generate class names for evaluation report.")
else:
    print("Model or test_loader not available for evaluation.")


# yolo 

In [ ]:
from ultralytics import YOLO
import os
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm

### Configuration

In [ ]:
YOLO_DATASET_ROOT = os.path.join("SignLanguage_Processed_Frames", "YOLO_Frames_Dataset")
TEST_SPLIT_DIR = os.path.join(YOLO_DATASET_ROOT, "test")

# Update this path to where your 'best.pt' is located from your training run
MODEL_PATH = "runs/slr_yolo_frame_cls_train/exp_frames/weights/best.pt" 

IMG_SIZE = 224 
BATCH_SIZE_TEST = 32 

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Evaluation Function

In [ ]:
def test_and_score_model(model_path, test_data_dir, img_size, batch_size):
    if not os.path.exists(model_path):
        print(f"ERROR: Trained model not found at {model_path}")
        return None
    if not os.path.exists(test_data_dir):
        print(f"ERROR: Test data directory not found at {test_data_dir}")
        return None

    model = YOLO(model_path)
    model.to(DEVICE)

    print(f"\n--- Evaluating Model on Test Set ---")
    print(f"Model: {model_path}")
    print(f"Test Data: {test_data_dir}")
    print(f"Image Size: {img_size}")
    print(f"Batch Size: {batch_size}")

    all_preds_indices = []
    all_true_labels_indices = []
    class_names_from_model = model.names
    class_to_idx = {name: idx for idx, name in class_names_from_model.items()}

    print("Collecting predictions on the test set...")
    for class_name in tqdm(os.listdir(test_data_dir), desc="Processing classes"):
        class_dir_path = os.path.join(test_data_dir, class_name)
        if not os.path.isdir(class_dir_path): continue
        if class_name not in class_to_idx: 
            print(f"Warning: Class '{class_name}' from test set not in model's classes. Skipping."); continue
        true_label_idx = class_to_idx[class_name]
        image_files = [os.path.join(class_dir_path, f) for f in os.listdir(class_dir_path) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files: continue

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i+batch_size]
            try:
                results = model.predict(source=batch_files, imgsz=img_size, verbose=False, device=DEVICE, stream=False)
                for res in results:
                    if hasattr(res, 'probs') and res.probs is not None:
                        pred_idx = res.probs.top1
                        all_preds_indices.append(pred_idx)
                        all_true_labels_indices.append(true_label_idx)
                    else:
                        print(f"Warning: No probabilities for an image in {class_name}. Result obj: {type(res)}")
            except Exception as e:
                print(f"Error during prediction for batch in {class_name}: {e}")

    if not all_true_labels_indices: print("No predictions were made."); return None

    num_model_classes = len(class_names_from_model)
    target_names_for_report_cm = [class_names_from_model[i] for i in range(num_model_classes)]
    unique_true_labels_indices = sorted(list(set(all_true_labels_indices)))
    target_names_report_only = [class_names_from_model[i] for i in unique_true_labels_indices]

    print("\n📊 Classification Report (per frame):")
    print(classification_report(
        all_true_labels_indices, 
        all_preds_indices, 
        labels=unique_true_labels_indices, 
        target_names=target_names_report_only, 
        digits=4, 
        zero_division=0
    ))

    accuracy = accuracy_score(all_true_labels_indices, all_preds_indices)
    f1_macro = f1_score(all_true_labels_indices, all_preds_indices, average='macro', zero_division=0)
    f1_weighted = f1_score(all_true_labels_indices, all_preds_indices, average='weighted', zero_division=0)

    print(f"\n🔍 Overall Test Accuracy (per frame): {accuracy:.4f}")
    print(f"🔍 Overall Test F1-Score (Macro, per frame): {f1_macro:.4f}")
    print(f"🔍 Overall Test F1-Score (Weighted, per frame): {f1_weighted:.4f}")

    cm = confusion_matrix(all_true_labels_indices, all_preds_indices, labels=list(range(num_model_classes)))
    plt.figure(figsize=(max(8, num_model_classes), max(6, num_model_classes * 0.8)))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names_for_report_cm,
                yticklabels=target_names_for_report_cm)
    plt.title("Confusion Matrix (Per Frame on Test Set)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig("confusion_matrix_yolo_frames_test.png")
    plt.show()

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "classification_report": classification_report(
            all_true_labels_indices, all_preds_indices, 
            labels=unique_true_labels_indices, target_names=target_names_report_only, 
            digits=4, zero_division=0, output_dict=True
        ),
        "confusion_matrix": cm
    }

### 5. Execute Evaluation

In [ ]:
results = test_and_score_model(
    model_path=MODEL_PATH,
    test_data_dir=TEST_SPLIT_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE_TEST
)
if results:
    print("\nEvaluation Finished.")
else:
    print("\nEvaluation could not be completed.")